# Code by Tanya Rajan and George Vojta.


(Additional thanks to Michael Cuna, Nadia Lucas and Jordan Rosenthal-Kay for their contribution. We appreciate them.)

In [1]:
# Importing packages #
from jax import numpy as jnp
import numpy as np
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
import math
from scipy.stats import norm
import scipy.integrate as integrate
import scipy
from random import choices
from numpy import matlib
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy.stats import mode
from scipy.sparse import identity
from scipy.sparse.linalg import inv as spinv
from scipy.sparse.linalg import spsolve as spsolve
from numpy.linalg import eig
import gurobipy as gp
from gurobipy import *
from linearmodels.iv import IV2SLS, IVGMM
import jax.numpy as jnp
import jax
import pyblp
from jax import core
np.random.seed(1)

In [17]:
### Import Data ###

dloc = "/Users/georgevojta/Dropbox/University of Chicago/Class Notes/Year2/IO2/problemsets/pset1/"
df = pd.read_csv(dloc+'rawdata.csv',sep = ',')

### DEFINE THESE LIKE THIS FOR PYBLP LATER (VERY RIGID PROGRAM) ###
'''df['demand_instruments0'] = df['z1']
df['demand_instruments1'] = df['z2']
df['demand_instruments2'] = df['z3']
df['demand_instruments3'] = df['z4']'''
df.head(10)

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2,z3,z4,shares
0,1,1,1.554708,3.0,1,0,0,0.237685,0.063116,0.034361,0.056459,0.239077
1,1,1,1.559811,5.0,1,0,0,0.002033,0.186073,0.326445,0.062091,0.072184
2,1,1,1.857186,3.0,1,1,0,0.164550,0.033308,0.075316,0.278736,0.000460
3,1,1,1.995438,5.0,1,0,1,0.485538,0.117025,0.345850,0.084891,0.587867
4,1,1,4.017987,7.5,1,0,1,0.582503,0.368032,0.652134,0.509693,0.033746
5,1,1,4.143667,10.0,1,0,1,0.564396,0.442613,0.169727,0.905535,0.021860
6,1,1,2.823407,10.0,0,0,1,0.275784,0.264170,0.504074,0.583465,0.036456
7,1,1,6.558325,12.0,0,0,1,1.183686,0.528303,0.315870,1.039286,0.000510
8,2,1,2.330368,5.0,1,0,0,0.316675,0.217231,0.294739,0.171832,0.014332
9,2,1,1.152118,5.0,1,1,0,0.253388,0.023140,0.490511,0.286931,0.245308


# Question 8 - Market 17 #

In [18]:
### Get Market ###
market1 = df[df["Market"]==17]
market1['Brand'] = np.array([1,1,2,2,3,3,3])

### FOR PYBLP - VERY RIGID SO NEED TO LABEL THINGS AS  'prices' AND 'market_ids'###
market1['prices'] = market1['Price']
market1['market_ids'] = market1['Market']

### Random Draw ###
market1['ξ'] = np.random.normal(0, 1, market1.count()[0])

### Observables ###
x = ["Constant","EngineSize","SportsBike","Brand2","Brand3"]
X = market1[x].to_numpy().reshape((market1.count()[0],len(x)))

X = np.hstack((market1['Price'].to_numpy().reshape((market1.count()[0],1)),X))
### Given Parameters ###
θ = np.array((-3,1,1,2,-1,1)).reshape((len(x)+1,1))

### Calc δ = \theta X + xi
market1['δ'] = np.dot(X,θ).reshape((market1.count()[0])) + market1['ξ']

market1['Brand1'] = 1 - market1['Brand2'] - market1['Brand3']

### Pred Share ###
market1['PredShare'] = np.exp(market1['δ'])/(1+np.sum(np.exp(market1['δ'])))
### Print for confirmation ###
market1.head(10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,Market,Constant,Price,EngineSize,SportsBike,Brand2,Brand3,z1,z2,z3,z4,shares,Brand,prices,market_ids,ξ,δ,Brand1,PredShare
113,17,1,0.920128,3.0,1,0,0,0.027602,0.028253,0.239770,0.270867,0.161315,1,0.920128,17,-0.836616,2.403000,1,0.098970
114,17,1,1.974534,5.0,1,0,0,0.219283,0.296879,0.494750,0.152504,0.014435,1,1.974534,17,0.778967,2.855365,1,0.155584
115,17,1,1.945479,3.0,1,1,0,0.150052,0.049772,0.035990,0.230106,0.001090,2,1.945479,17,1.363267,0.526829,0,0.015160
116,17,1,1.860885,5.0,1,1,0,0.387576,0.301157,0.274545,0.099373,0.037353,2,1.860885,17,0.160075,1.577420,0,0.043347
117,17,1,1.983073,6.5,0,0,1,0.071480,0.033719,0.332783,0.103803,0.470942,3,1.983073,17,-0.743913,1.806869,0,0.054527
118,17,1,3.898499,10.0,1,0,1,0.052336,0.601297,0.525480,0.560475,0.081604,3,3.898499,17,-0.662154,1.642348,0,0.046255
119,17,1,2.864794,10.0,0,0,1,0.152532,0.060189,0.814082,0.517434,0.212910,3,2.864794,17,0.760759,4.166377,0,0.577205


In [4]:
xvars = ['Constant','EngineSize', 'SportsBike', 'Brand2', 'Brand3']
x = market1[xvars].to_numpy().reshape((market1.count()[0],len(xvars)))
xi =  market1['ξ'].to_numpy().reshape((7,1))
pguess = np.ones(7).reshape((7,1))
X = np.hstack((pguess,x))

### Theta should be price coef, then xvar coefs ###
def price_contract(theta, x, xi, pguess = np.ones(7).reshape((7,1))):
    ### Put prices in front ###
    p = pguess
    diff = 1000
    while diff > .00001:  
        out = f(theta, x, xi, p)
        
        prime = -(jax.jacfwd(lambda P: f(θ,x,xi,P))(p)).reshape((pguess.shape[0],pguess.shape[0]))
        pnew = p + jnp.dot(jnp.linalg.inv(prime),out)
        diff = max(abs(p-pnew))
        
        p = pnew
        
    ### if you reset alpha to -theta[0] then you need to return -p, no idea why ###
    return p

def f(theta, x, xi, pguess = np.ones(7).reshape((7,1))):
    ### Create delta ###
    #### Alpha should be first component ###
    alpha = theta[0]
    delta = alpha*pguess + np.dot(x,theta[1:]) + xi
    ### Get Shares ###
    numerators = jnp.exp(delta)
    denominator = 1+numerators.sum()
    
    sHat = jnp.divide(numerators, denominator)
    ### ownership mat
    D = np.eye(X.shape[0])
    
    
    ### Construct Jacobian ###
    J = alpha*jnp.dot(sHat, sHat.T)
    Jkill = jnp.multiply(jnp.eye((sHat.shape[0])) ,jnp.diagonal(alpha*jnp.dot(sHat, sHat.T)))
    Jdiag = jnp.multiply(jnp.eye((sHat.shape[0])) ,jnp.diagonal(-alpha*jnp.dot(sHat, (1-sHat).T)))
    
    J = J - Jkill + Jdiag
    
    for i in range(0,len(sHat)):
    ### Kill the crosses if not the same brand ###
        for j in range(0,len(sHat)):
            if i != j:
                ### Sets owndership to 1 if same brand ###
                if (X[i][4] == X[j][4]) and (X[i][5] == X[j][5]):
                    D[i,j] = 1
                    pass
                else:
                    pass
                
    D = jnp.asarray(D)
    out = -((jnp.dot(jnp.linalg.inv(jnp.multiply(D,J)),sHat)) - pguess)

    return jnp.asarray(out) 

In [5]:
p = price_contract(θ,x,xi)
print(p)

[[0.39907336]
 [0.3990733 ]
 [0.3417734 ]
 [0.3417734 ]
 [1.7583168 ]
 [1.7583166 ]
 [1.7583166 ]]


# Question 9

In [7]:
def problogit(u):
    numerator = np.exp(u)
    denominator = numerator.sum()
    return numerator / (1+denominator)


def ubar(δ, σ, X, zeta,J):
    u = δ + np.dot((X*zeta), σ)
    return np.array(u).reshape((1,J))

def sHat(δ, σ, x, zeta, J, I, fullmat = False):
    choose = np.zeros((I,J))
    for i in range(0,I):
        choose[i,:]=  problogit(ubar(δ, σ, x, zeta[i,:],J))
    
    share = np.mean(choose,axis = 0)
    if fullmat == False:
        return share.reshape((J,1))
    if fullmat == True:
        return choose

In [8]:
### TESTS ###

x = ["Price", "Constant", "EngineSize","SportsBike","Brand2","Brand3"]
X = market1[x].to_numpy().reshape((market1.count()[0],len(x)))
θ = np.array((-3,1,1,2,-1,1)).reshape((len(x),1))
I = 20
J = 3

Xrun = X[0:J,:]

zeta = np.random.normal(0,1,I*len(x)).reshape((I,len(x)))
sigma = np.zeros((len(x),1))
delta = np.zeros((J,1))

print("TEST 1: ", sHat(delta, sigma, Xrun, zeta, J, I))

I = 20
J = 3

Xrun = X[0:J,:]

zeta = np.random.normal(0,1,I*len(x)).reshape((I,len(x)))
sigma = np.zeros((len(x),1))
delta = np.array([40, 20, 20]).reshape(J,1)

print("TEST 2: ", sHat(delta, sigma, Xrun, zeta, J, I))



zeta = np.random.normal(0,1,I*len(x)).reshape((I,len(x)))
sigma = np.array([.1, 0, .1, 0, 0, 0]).reshape(len(x),1)
delta = np.zeros((J,1))
Xzero = np.zeros((J,len(x)))

print("TEST 3: ", sHat(delta, sigma, Xzero, zeta, J, I))

print("TEST 4 (No Zeroes): ", sHat(delta, sigma, Xrun, zeta, J, I))

TEST 1:  [[0.25]
 [0.25]
 [0.25]]
TEST 2:  [[9.99999996e-01]
 [2.06115361e-09]
 [2.06115361e-09]]
TEST 3:  [[0.25]
 [0.25]
 [0.25]]
TEST 4 (No Zeroes):  [[0.24311867]
 [0.24929794]
 [0.24628762]]


# Question 10

In [9]:
def shareFP(share, delta_start, tol, iterate, X, I, J, sig, xi):
    diff = 100000000
    
    ### We will loop through deltas ###
    delta_working = delta_start
    iteration = 1
    
    ### While intolerable ###
    while (diff > tol) and (iteration < iterate):
        delta_last = delta_working
        
        ### Predict Shares ###
        shat = sHat(delta_working, sig, X, zeta, J, I).reshape((J,1))
    
        ### BLP Comp Page 7 (Contraction Mapping) ###
        delta_working = delta_working + np.log(share) - np.log(shat)
        
        ### Lots of different ways to set distances ###
        diff = np.max(np.abs(delta_working - delta_last))
        iteration += 1
    return delta_working

In [10]:
### TEST IT OUT BABY ###

delta = np.array([1.4, 2, 1.8]).reshape(J,1)
s = sHat(delta, sigma, Xrun, zeta, J, I)
print("Shares are", s, "Sum is", s.sum())
zeta = np.random.normal(0,1,I*len(x)).reshape((I,len(x)))
sigma = np.zeros((len(x),1))
deltag = np.array([1, 1, 1]).reshape(J,1)

tolerance = .000000000000000000000001 

print("Deltas contract to", shareFP(s,deltag,tolerance, 50000, Xrun,20,J,sigma,zeta))

Shares are [[0.21787215]
 [0.39564278]
 [0.32613998]] Sum is 0.9396549041821096
Deltas contract to [[1.28382872]
 [1.88043204]
 [1.687247  ]]


# Question 14 (Market 17 Only)

In [19]:
### Start by doing the linear and non-linear components ###
### Absorbing the Brand Fixed Effects###
product_form_linear = pyblp.Formulation('prices + EngineSize + SportsBike', absorb = 'C(Brand)')

### Then the components that add on as random coefficients ###
product_form_shocks = pyblp.Formulation('0 + prices + EngineSize')

### This is just some dual thing to load into PYBLP###
PF = (product_form_linear,product_form_shocks)

In [20]:
### Set up Agents ###

### Draw set of 1000 ###
I = 2000
x = ['Constant', 'Price', 'EngineSize', 'SportsBike']

### Make individual data frame ###
### Made up of individual their price shocks and engine shocks ###
dfI = pd.DataFrame({'I': np.arange(I), 'alpha_shock': np.random.normal(0,1,I), 'engine_shock': np.random.normal(0,1,I)})
dfI['weights'] = 1/I

### NEED MARKET_IDS BECAUSE PYBLP IS RIGID ###
dfI['market_ids'] = 17


AF = pyblp.Formulation('0 + alpha_shock + engine_shock')

In [21]:
### Only sample some, why? I don't know ###
Sample = int(.1*I)

problem = pyblp.Problem(
    PF,
    market1,
    AF,
    dfI,
    ### I just did even weight integration ###
    integration = pyblp.Integration('monte_carlo',Sample)
)
problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T    N    I    K1    K2    D    MD    ED 
---  ---  ---  ----  ----  ---  ----  ----
 1    7   200   3     2     2    2     1  

Formulations:
       Column Indices:              0            1            2     
-----------------------------  -----------  ------------  ----------
 X1: Linear Characteristics      prices      EngineSize   SportsBike
X2: Nonlinear Characteristics    prices      EngineSize             
       d: Demographics         alpha_shock  engine_shock            


Dimensions:
 T    N    I    K1    K2    D    MD    ED 
---  ---  ---  ----  ----  ---  ----  ----
 1    7   200   3     2     2    2     1  

Formulations:
       Column Indices:              0            1            2     
-----------------------------  -----------  ------------  ----------
 X1: Linear Characteristics      prices      EngineSize   SportsBike
X2: Nonlinear Characteristics    prices      EngineSize             
       d: Demographics         alpha_shock  engine_shock            

In [22]:
### Guessing that it's diagonal and  setting the coefficient infront of interacting zeta*X terms is 1)
SIG = np.array([.1,0,0,.1]).reshape((2,2))
PI = np.array([1,0,0,1]).reshape((2,2))
results = problem.solve(sigma = SIG, pi = PI)

Solving the problem ...

Nonlinear Coefficient Initial Values:
  Sigma:       prices       EngineSize    |     Pi:       alpha_shock   engine_shock 
----------  -------------  -------------  |  ----------  -------------  -------------
  prices    +1.000000E-01                 |    prices    +1.000000E+00  +0.000000E+00
EngineSize  +0.000000E+00  +1.000000E-01  |  EngineSize  +0.000000E+00  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
  Sigma:       prices       EngineSize    |     Pi:       alpha_shock   engine_shock 
----------  -------------  -------------  |  ----------  -------------  -------------
  prices    +0.000000E+00                 |    prices        -INF       +0.000000E+00
EngineSize  +0.000000E+00  +0.000000E+00  |  EngineSize  +0.000000E+00      -INF     

Nonlinear Coefficient Upper Bounds:
  Sigma:       prices       EngineSize    |     Pi:       alpha_shock   engine_shock 
----------  -------------  -------------  |  ----------  -------------  -------------
  p

In [ ]:
elasticities = results.compute_elasticities().round(decimals = 2)
print(elasticities)

# Question 14 All Markets

In [ ]:
### Get Market ###
df['Brand'] = 1
df.loc[df['Brand2']==1, 'Brand'] = 2
df.loc[df['Brand3']==3, 'Brand'] = 3

### FOR PYBLP - VERY RIGID SO NEED TO LABEL THINGS AS  'prices' AND 'market_ids'###
df['prices'] = df['Price']
df['market_ids'] = df['Market']
df['demand_instruments0'] = df['z1']
df['demand_instruments1'] = df['z2']
df['demand_instruments2'] = df['z3']
df['demand_instruments3'] = df['z4']


### Observables ###
x = ["Constant","EngineSize","SportsBike","Brand2","Brand3"]
X = df[x].to_numpy().reshape((df.count()[0],len(x)))

X = np.hstack((df['Price'].to_numpy().reshape((df.count()[0],1)),X))
df

In [ ]:
### Start by doing the linear and non-linear components ###
### Absorbing the Brand Fixed Effects###
product_form_linear_all = pyblp.Formulation('1 + prices + EngineSize + SportsBike', absorb = 'C(Brand)')

### Then the components that add on as random coefficients ###
product_form_shocks_all = pyblp.Formulation('0 + prices + EngineSize')

### This is just some dual thing to load into PYBLP###
PFall = (product_form_linear_all,product_form_shocks_all)


### Set up Agents ###
### Draw set of 1000 ###
I = 150000
markets = np.kron(np.ones(1000),np.arange(151)[1:]).astype(int)

x = ['Constant', 'Price', 'EngineSize', 'SportsBike']

### Make individual data frame ###
### Made up of individual their price shocks and engine shocks ###
dfI = pd.DataFrame({'I': np.arange(I), 'alpha_shock': np.random.normal(0,1,I), 'engine_shock': np.random.normal(0,1,I), 'market_ids':markets})
dfI['weights'] = 1/(I/150)



AFall = pyblp.Formulation('0 + alpha_shock + engine_shock')
dfI.head(10)

In [ ]:
### Only sample some, why? I don't know ###
Sample = int(I/150)

problem = pyblp.Problem(
    PFall,
    df,
    AFall,
    dfI,
    ### I just did even weight integration ###
    integration = pyblp.Integration('monte_carlo',Sample)
)


### Guessing that it's diagonal and  setting the coefficient infront of interacting zeta*X terms is 1)
SIG = np.array([1,0,0,1]).reshape((2,2))
PI = np.array([1,0,0,1]).reshape((2,2))
results = problem.solve(sigma = SIG, pi = PI)

In [ ]:
elasticities = results.compute_elasticities().round(decimals = 2)
print(np.round(elasticities[17*7+1:17*7+8,0:6]))